# Qulacs & openfermion & psi4 で VQE

必要なもの

- qulacs
- openfermion
- openfermion-psi4
- psi4
- scipy
- numpy

In [ ]:
import sys, os
import qulacs

from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.utils import Grid
from openfermion.linalg import get_sparse_operator
from openfermion.chem import MolecularData
from openfermionpsi4 import run_psi4

from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt

## ハミルトニアンを作る

In [ ]:
basis = "sto-3g"
multiplicity = 1
charge = 0
trotter_step = 1
distance  = 0.977
geometry = [["H", [0,0,0]],["H", [0,0,distance]]]
description  = "tmp"
molecule = MolecularData(geometry, basis, multiplicity, charge, description)
molecule = run_psi4(molecule,run_scf=1,run_fci=1)

n_qubit = molecule.n_qubits
n_electron = molecule.n_electrons

In [8]:
from openfermion.chem import MolecularData

# Set parameters to make a simple molecule.
diatomic_bond_length = .7414
geometry = [('Ti', (2.866, 0, 0)), ('O', (3.732, 0.5, 0)), ('O', (2, -0.5, 0))]
basis = 'sto-3g'
multiplicity = 1
charge = 0
description = str(diatomic_bond_length)

# Make molecule and print out a few interesting facts about it.
molecule = MolecularData(geometry, basis, multiplicity,
                         charge, description)
print('Molecule has automatically generated name {}'.format(
    molecule.name))
print('Information about this molecule would be saved at:\n{}\n'.format(
    molecule.filename))
print('This molecule has {} atoms and {} electrons.'.format(
    molecule.n_atoms, molecule.n_electrons))
for atom, atomic_number in zip(molecule.atoms, molecule.protons):
    print('Contains {} atom, which has {} protons.'.format(
        atom, atomic_number))

Molecule has automatically generated name O2-Ti1_sto-3g_singlet_0.7414
Information about this molecule would be saved at:
/home/yourn/.conda/envs/qe/lib/python3.9/site-packages/openfermion/testing/data/O2-Ti1_sto-3g_singlet_0.7414

This molecule has 3 atoms and 38 electrons.
Contains O atom, which has 8 protons.
Contains O atom, which has 8 protons.
Contains Ti atom, which has 22 protons.


### ハミルトニアンを変換・表示する

In [ ]:
from openfermion.chem import geometry_from_pubchem

methane_geometry = geometry_from_pubchem('titanium-dioxide')
print(methane_geometry)


In [10]:
# Set molecule parameters.
basis = 'sto-3g'
multiplicity = 1
bond_length_interval = 0.1
n_points = 25
# Generate molecule at different bond lengths.
hf_energies = []
fci_energies = []
bond_lengths = []
for point in range(3, n_points + 1):
    bond_length = bond_length_interval * point
    bond_lengths += [bond_length]
    description = str(round(bond_length,2))
    print(description)
    geometry = [('Ti', (2.866, 0, 0)), ('O', (3.732, 0.5, 0)), ('O', (2, -0.5, 0))]
    molecule = MolecularData(
        geometry, basis, multiplicity, description=description)
    
    # Load data.
    molecule.load()

    # Print out some results of calculation.
    print('\nAt bond length of {} angstrom, molecular tio2 has:'.format(
        bond_length))
    print('Hartree-Fock energy of {} Hartree.'.format(molecule.hf_energy))
    print('MP2 energy of {} Hartree.'.format(molecule.mp2_energy))
    print('FCI energy of {} Hartree.'.format(molecule.fci_energy))
    print('Nuclear repulsion energy between protons is {} Hartree.'.format(
        molecule.nuclear_repulsion))
    for orbital in range(molecule.n_orbitals):
        print('Spatial orbital {} has energy of {} Hartree.'.format(
            orbital, molecule.orbital_energies[orbital]))
    hf_energies += [molecule.hf_energy]
    fci_energies += [molecule.fci_energy]

# Plot.
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(0)
plt.plot(bond_lengths, fci_energies, 'x-')
plt.plot(bond_lengths, hf_energies, 'o-')
plt.ylabel('Energy in Hartree')
plt.xlabel('Bond length in angstrom')
plt.show()

0.3


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/yourn/.conda/envs/qe/lib/python3.9/site-packages/openfermion/testing/data/O2-Ti1_sto-3g_singlet_0.3.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_ground_state, get_sparse_operator
import numpy
import scipy
import scipy.linalg

# Load saved file for LiH.
diatomic_bond_length = 0.74
geometry = [('Ti', (2.866, 0, 0)), ('O', (3.732, 0.5, 0)), ('O', (2, -0.5, 0))]
basis = 'sto-3g'
multiplicity = 1

# Set Hamiltonian parameters.
active_space_start = 1
active_space_stop = 3

# Generate and populate instance of MolecularData.
molecule = MolecularData(geometry, basis, multiplicity, description=".74")
molecule.load()

# Get the Hamiltonian in an active space.
molecular_hamiltonian = molecule.get_molecular_hamiltonian(
    occupied_indices=range(active_space_start),
    active_indices=range(active_space_start, active_space_stop))

# Map operator to fermions and qubits.
fermion_hamiltonian = get_fermion_operator(molecular_hamiltonian)
qubit_hamiltonian = jordan_wigner(fermion_hamiltonian)
qubit_hamiltonian.compress()
print('The Jordan-Wigner Hamiltonian in canonical basis follows:\n{}'.format(qubit_hamiltonian))

# Get sparse operator and ground state energy.
sparse_hamiltonian = get_sparse_operator(qubit_hamiltonian)
energy, state = get_ground_state(sparse_hamiltonian)
print('Ground state energy before rotation is {} Hartree.\n'.format(energy))

# Randomly rotate.
n_orbitals = molecular_hamiltonian.n_qubits // 2
n_variables = int(n_orbitals * (n_orbitals - 1) / 2)
numpy.random.seed(1)
random_angles = numpy.pi * (1. - 2. * numpy.random.rand(n_variables))
kappa = numpy.zeros((n_orbitals, n_orbitals))
index = 0
for p in range(n_orbitals):
    for q in range(p + 1, n_orbitals):
        kappa[p, q] = random_angles[index]
        kappa[q, p] = -numpy.conjugate(random_angles[index])
        index += 1

    # Build the unitary rotation matrix.
    difference_matrix = kappa + kappa.transpose()
    rotation_matrix = scipy.linalg.expm(kappa)

    # Apply the unitary.
    molecular_hamiltonian.rotate_basis(rotation_matrix)

# Get qubit Hamiltonian in rotated basis.
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
qubit_hamiltonian.compress()
print('The Jordan-Wigner Hamiltonian in rotated basis follows:\n{}'.format(qubit_hamiltonian))

# Get sparse Hamiltonian and energy in rotated basis.
sparse_hamiltonian = get_sparse_operator(qubit_hamiltonian)
energy, state = get_ground_state(sparse_hamiltonian)
print('Ground state energy after rotation is {} Hartree.'.format(energy))


In [ ]:
fermionic_hamiltonian = get_fermion_operator(molecule.get_molecular_hamiltonian())
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)

In [ ]:
print(jw_hamiltonian)

### ハミルトニアンを qulacs ハミルトニアンに変換する

In [ ]:
from qulacs import Observable
from qulacs.observable import create_observable_from_openfermion_text

In [ ]:
qulacs_hamiltonian = create_observable_from_openfermion_text(str(jw_hamiltonian))

## ansatz を構成する

In [ ]:
from qulacs import QuantumState, QuantumCircuit
from qulacs.gate import CZ, RY, RZ, merge

depth = n_qubit

In [ ]:
def he_ansatz_circuit(n_qubit, depth, theta_list):
    """he_ansatz_circuit
    Retunrs hardware efficient ansatz circuit.

    Args:
        n_qubit (:class:`int`):
            the number of qubit used (equivalent to the number of fermionic modes)
        depth (:class:`int`):
            depth of the circuit.
        theta_list (:class:`numpy.ndarray`):
            rotation angles.
    Returns:
        :class:`qulacs.QuantumCircuit`
    """
    circuit = QuantumCircuit(n_qubit)
    
    for d in range(depth):
        for i in range(n_qubit):
            circuit.add_gate(merge(RY(i, theta_list[2*i+2*n_qubit*d]), RZ(i, theta_list[2*i+1+2*n_qubit*d])))
        for i in range(n_qubit//2):
            circuit.add_gate(CZ(2*i, 2*i+1))
        for i in range(n_qubit//2-1):
            circuit.add_gate(CZ(2*i+1, 2*i+2))
    for i in range(n_qubit):
        circuit.add_gate(merge(RY(i, theta_list[2*i+2*n_qubit*depth]), RZ(i, theta_list[2*i+1+2*n_qubit*depth])))
    
    return circuit

## Hartree Fock エネルギーのチェック

In [ ]:
input_state = QuantumState(n_qubit)
input_state.set_computational_basis(int('0b'+'0'*(n_qubit - n_electron)+'1'*(n_electron),2))

In [ ]:
qulacs_hamiltonian.get_expectation_value(input_state)

In [ ]:
molecule.hf_energy

## VQE の cost function

In [ ]:
def cost(theta_list):
    input_state.set_computational_basis(int('0b'+'0'*(n_qubit - n_electron)+'1'*(n_electron),2))
    circuit = he_ansatz_circuit(n_qubit, depth, theta_list)
    circuit.update_quantum_state(input_state)
    return qulacs_hamiltonian.get_expectation_value(input_state)

## 初期パラメータセット

In [ ]:
init_theta_list = np.random.random(2*n_qubit*(depth+1))*1e-1
init_theta_list

## 最適化

In [ ]:
method = "BFGS"
options = {"disp": True, "maxiter": 50, "gtol": 1e-6}
opt = minimize(cost, init_theta_list, method = method)

In [ ]:
opt.x

In [ ]:
print(opt.fun, float(molecule.fci_energy))